# Fine Tuning in Torch


In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModel, TrainingArguments, AutoModelForSequenceClassification, Trainer
from datasets import load_dataset
import torch as tf

In [2]:
# importing data and defining checkpoints

dataset = load_dataset ('glue', 'mrpc')
checkpoint = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained (checkpoint)
data_collector_padding = DataCollatorWithPadding (tokenizer=tokenizer)
training_args = TrainingArguments ("test-trainer")
model = AutoModelForSequenceClassification.from_pretrained (checkpoint)

print (dataset)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [3]:
train_dataset = dataset['train']
print (len (train_dataset))
train_dataset[:3]

3668


{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .'],
 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale ."],
 'label': [1, 0, 1],
 'idx': [0, 1, 2]}

In [4]:
# creating tokenizer funciton to to tokenize the dataset

def tockeinzer_df (dataset_to_tockenize):
    return  tokenizer (dataset_to_tockenize['sentence1'], dataset_to_tockenize['sentence2'], truncation=True) 

In [5]:
tain_data_tockenized =  train_dataset.map (tockeinzer_df)

In [6]:
tain_data_tockenized[:3].keys()

dict_keys(['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

In [7]:
valid_dataset = dataset['validation']
print (valid_dataset)
valid_dataset_tok = valid_dataset.map (tockeinzer_df)
print (valid_dataset_tok)
valid_dataset_tok.column_names

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 408
})
Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 408
})


['sentence1',
 'sentence2',
 'label',
 'idx',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [36]:
trainer = Trainer(

    model, 
    training_args, 
    train_dataset = tain_data_tockenized, 
    eval_dataset = valid_dataset_tok, 
    data_collator = data_collector_padding, 
    tokenizer= tokenizer,

)

In [23]:

trainer.train()

c:\Users\Z004RJZU\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6332, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}
{'loss': 0.4943, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}
{'train_runtime': 821.5063, 'train_samples_per_second': 13.395, 'train_steps_per_second': 1.676, 'train_loss': 0.5141200756109012, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.5141200756109012, metrics={'train_runtime': 821.5063, 'train_samples_per_second': 13.395, 'train_steps_per_second': 1.676, 'train_loss': 0.5141200756109012, 'epoch': 3.0})

## Evaluation on Validation set

In [24]:
prediction = trainer.predict (valid_dataset_tok)
print (prediction.predictions.shape)

  0%|          | 0/51 [00:00<?, ?it/s]

(408, 2)


In [30]:

prediction.label_ids

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,

In [31]:
import numpy as np

preds = np.argmax(prediction.predictions, axis=-1)
preds

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,

In [35]:
import evaluate

metric = evaluate.load("glue",'mrpc')
metric.compute (predictions=preds, references=prediction.label_ids)

{'accuracy': 0.8235294117647058, 'f1': 0.8754325259515572}

In [8]:
import evaluate
import numpy as np

def evaluate_def(eval_prediction):
    metric = evaluate.load ('glue', 'mrpc')
    logits, labels = eval_prediction
    prediction = np.argmax (logits, axis=-1)
    return metric.compute (predictions=prediction, references=labels)

training_args = TrainingArguments ('test-trainer', evaluation_strategy='epoch')
model = AutoModelForSequenceClassification.from_pretrained (checkpoint, num_labels=2)

model.to (tf.device('cuda'))


trainer = Trainer (
    model,
    training_args,
    train_dataset = tain_data_tockenized,
    eval_dataset  = valid_dataset_tok,
    data_collator=data_collector_padding,
    tokenizer=tokenizer,
    compute_metrics=evaluate_def
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Z004RJZU\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/51 [00:00<?, ?it/s]

NameError: name 'np' is not defined

In [44]:
model.to (tf.device('cpu'))

trainer.train()

  0%|          | 0/1377 [00:00<?, ?it/s]

KeyError: 'exp_avg_sq'